# MovieLens + TwoTower modelでembeddingを作るデモ

このノートブックはMovieLensとTwoTowerモデルで、ベクトル検索に用いるembeddingsを作るデモのJupyter Labノートブックです。

実行するにはNVIDIA製のGPUと、NVIDIA製のDockerイメージが必要です。ローカルで動作環境を整えるのは困難を極めるのでお勧めしません。
このノートブックの動作確認はRTX 4070を積んだWindows 11デスックトップマシンで、Docker Desktop for Windowsを用いました。

このファイルをローカルDockerで開くには、このファイルがあるディレクトリで以下のコマンドを実行します。
このコマンドはこのディレクトリにある `run-merlintf-jupyterlab.sh` でも代用できます。

```
docker run -it --rm --gpus all 
    -p 8000:8000 -p 8001:8001 -p 8002:8002 -p 8888:8888 
    -v .://workspace/data/ --ipc=host 
    -w //workspace/data 
    nvcr.io/nvidia/merlin/merlin-tensorflow:nightly 
    jupyter lab --allow-root --ip='0.0.0.0'
```

端末に以下のようなメッセージが表示されたら、`127.0.0.1:8888` のURLをホストマシンのブラウザで開くと、
Jupyter Labにアクセスできます。

```
[C 2023-11-01 06:05:58.736 ServerApp]

    To access the server, open this file in a browser:
        file:///tmp/.jupyter/jpserver-1-open.html
    Or copy and paste one of these URLs:
        http://70439a8d96ae:8888/lab?token=7e5d667023a6d0e5c78246efb6e0423c84a3edf8fd5ea683
        http://127.0.0.1:8888/lab?token=7e5d667023a6d0e5c78246efb6e0423c84a3edf8fd5ea683
```

Jupyter Labにアクセスしたら右側の領域から、本ファイル movielens+twotowermodel.ipynb をダブルクリックして開いてください。
本ノートブックを手元のJupyter Labで表示できます。
あとはPythonコードのセルを順番に実行することで動作確認ができます。



## インポート

まずは必要なモジュールをインポートします。
完了まで少々時間がかかること、また環境によってはログや警告メッセージが表示されるので、留意してください。

In [1]:
import os

from merlin.datasets.entertainment import get_movielens
from merlin.schema.tags import Tags
import merlin.models.tf as mm
from merlin.models.utils.dataset import unique_rows_by_features
from merlin.io.dataset import Dataset

import tensorflow as tf

2023-11-01 07:10:38.264126: I tensorflow/core/platform/cpu_feature_guard.cc:183] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/usr/local/lib/python3.10/dist-packages/merlin/dtypes/mappings/torch.py:43: UserWarning: PyTorch dtype mappings did not load successfully due to an error: No module named 'torch'
  warn(f"PyTorch dtype mappings did not load successfully due to an error: {exc.msg}")


2023-11-01 07:10:40.001797: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:999] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-01 07:10:40.002147: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:999] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-01 07:10:40.002172: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:999] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


[INFO]: sparse_operation_kit is imported


[SOK INFO] Import /usr/local/lib/python3.10/dist-packages/merlin_sok-1.2.0-py3.10-linux-x86_64.egg/sparse_operation_kit/lib/libsok_experiment.so
[SOK INFO] Import /usr/local/lib/python3.10/dist-packages/merlin_sok-1.2.0-py3.10-linux-x86_64.egg/sparse_operation_kit/lib/libsok_experiment.so
[SOK INFO] Initialize finished, communication tool: horovod


2023-11-01 07:10:40.295838: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:999] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-01 07:10:40.295913: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:999] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-01 07:10:40.295939: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:999] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-01 07:10:40.307554: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:999] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-01 07:10:40.307576: I tensorflow/core/co

## MovieLensデータの準備

MovieLensのデータを必要があればダウンロード、展開して読み込み、Merlin Tensorflowで利用可能な形に整形します。

データは _data/ ディレクトリ内に配置されます。
また中間ファイルが categories/ ディレクトリに作成されます。

読み込んだデータは訓練用の `train` と 検証の `valid` の2つのデータセットに 8:2 の割合で分けられています。

In [2]:
DATADIR = os.environ.get("INPUT_DATA_DIR", os.path.expanduser("./_data/"))

In [3]:
train, valid = get_movielens(variant="ml-1m", path=DATADIR)

データは映画作品を評価したユーザーの情報(user)と、評価された映画作品(item)の情報が非正規化された、1つのテーブルになっています。

また各カラムにはタグで情報づけられています。
`Tags.USER` はユーザーの情報、 `Tags.ITEM` はアイテムの情報、 `Tags.ID` はユーザーとアイテムそれぞれのIDに付けられます。
Tensorflowはこのタグを識別してモデルの作成に利用します。

次のコードはそのデータのスキーマから、ユーザーに関するものだけを抜き出し表示します。

In [4]:
train.schema.select_by_tag(Tags.USER)

,name,tags,dtype,is_list,is_ragged,properties.num_buckets,properties.freq_threshold,properties.max_size,properties.cat_path,properties.embedding_sizes.cardinality,properties.embedding_sizes.dimension,properties.domain.min,properties.domain.max,properties.domain.name
0,TE_userId_rating,"(Tags.CONTINUOUS, Tags.USER)","DType(name='float64', element_type=<ElementTyp...",False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,TE_age_rating,"(Tags.CONTINUOUS, Tags.USER)","DType(name='float64', element_type=<ElementTyp...",False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,TE_gender_rating,"(Tags.CONTINUOUS, Tags.USER)","DType(name='float64', element_type=<ElementTyp...",False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,TE_occupation_rating,"(Tags.CONTINUOUS, Tags.USER)","DType(name='float64', element_type=<ElementTyp...",False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,TE_zipcode_rating,"(Tags.CONTINUOUS, Tags.USER)","DType(name='float64', element_type=<ElementTyp...",False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,userId,"(Tags.ID, Tags.USER, Tags.CATEGORICAL)","DType(name='int32', element_type=<ElementType....",False,False,NaN,0.0,0.0,.//categories/unique.userId.parquet,6043.0,210.0,0.0,6042.0,userId


次のコードは映画作品に関するスキーマだけを抜き出します。

なおこのデモで利用しているシンプルなTwo Towerでは、ユーザーと映画作品を繋ぐ「レーティング」のような情報は取り扱えません。
それを取り扱えるようにする研究として [IntTower](https://github.com/archersama/IntTower/) が存在します。

In [5]:
train.schema.select_by_tag(Tags.ITEM)

,name,tags,dtype,is_list,is_ragged,properties.num_buckets,properties.freq_threshold,properties.max_size,properties.cat_path,properties.embedding_sizes.cardinality,properties.embedding_sizes.dimension,properties.domain.min,properties.domain.max,properties.domain.name,properties.value_count.min,properties.value_count.max
0,TE_movieId_rating,"(Tags.CONTINUOUS, Tags.ITEM)","DType(name='float64', element_type=<ElementTyp...",False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,movieId,"(Tags.ID, Tags.ITEM, Tags.CATEGORICAL)","DType(name='int32', element_type=<ElementType....",False,False,NaN,0.0,0.0,.//categories/unique.movieId.parquet,3675.0,159.0,0.0,3674.0,movieId,NaN,NaN
2,genres,"(Tags.ITEM, Tags.CATEGORICAL)","DType(name='int32', element_type=<ElementType....",True,True,NaN,0.0,0.0,.//categories/unique.genres.parquet,21.0,16.0,0.0,20.0,genres,1.0,6.0


## モデルの構築と学習

データの確認が終わると、モデルの構築と学習に入れます。

Two Towerモデルは、ユーザタワーとアイテムタワー、2つの多層パーセプトロンを必要とします。
1つ1つのユーザーとアイテムのペアが、それぞれのタワーで空間上の近いところに射影≒配置されるように学習することで、embeddingのための変換を獲得します。

次のコードではそれぞれのタワーを128次元と64次元の2層のパーセプトロンとして作成し、
ユーザー側のスキーマとアイテム側のスキーマを紐づけています。

その後に2つのタワー≒パーセプトロンを組み合わせてTwo Towerモデルを構築します。

In [6]:
tower_dim = 64

user_schema = train.schema.select_by_tag(Tags.USER)
user_inputs = mm.InputBlockV2(user_schema)
query = mm.Encoder(user_inputs, mm.MLPBlock([128, tower_dim], no_activation_last_layer=True))

item_schema = train.schema.select_by_tag(Tags.ITEM)
item_inputs = mm.InputBlockV2(item_schema)
candidate = mm.Encoder(item_inputs, mm.MLPBlock([128, tower_dim], no_activation_last_layer=True))

In [7]:
model = mm.TwoTowerModelV2(query, candidate)

Two Towerモデルが構築できたら学習です。

GPUを用いても少々時間がかかるので、気長に待ちましょう。
実行中には動作ログや警告が出力されることがあります。

また稀に失敗することがあるのでその場合は再実行するか、
さらに失敗する用ならJupyter Labのカーネルを再起動して、最初の手順からやり直してみてください。

In [8]:
model.compile(optimizer="adam", run_eagerly=False, metrics=[mm.RecallAt(10), mm.NDCGAt(10)])
model.fit(train, validation_data=valid, batch_size=4096, epochs=2)

2023-11-01 07:10:44.041181: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:655] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


Epoch 1/2


2023-11-01 07:10:44.280998: I tensorflow/core/common_runtime/executor.cc:1209] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


196/196 [==============================] - ETA: 0s - loss: 8.2099 - recall_at_10: 0.0051 - ndcg_at_10: 0.0026 - regularization_loss: 0.0000e+00 - loss_batch: 8.2061

2023-11-01 07:10:56.685577: I tensorflow/core/common_runtime/executor.cc:1209] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


196/196 [==============================] - 15s 42ms/step - loss: 8.2099 - recall_at_10: 0.0052 - ndcg_at_10: 0.0026 - regularization_loss: 0.0000e+00 - loss_batch: 8.2004 - val_loss: 8.1131 - val_recall_at_10: 0.0088 - val_ndcg_at_10: 0.0044 - val_regularization_loss: 0.0000e+00 - val_loss_batch: 7.9468
Epoch 2/2
196/196 [==============================] - 7s 35ms/step - loss: 8.0682 - recall_at_10: 0.0101 - ndcg_at_10: 0.0050 - regularization_loss: 0.0000e+00 - loss_batch: 8.0588 - val_loss: 8.0294 - val_recall_at_10: 0.0119 - val_ndcg_at_10: 0.0060 - val_regularization_loss: 0.0000e+00 - val_loss_batch: 7.8682


## モデルの評価

次に学習を完了したモデルの評価を行います。

評価はモデルから近似トップKを得るモデルを作成し、そのトップKモデルに対して検証用データを適用することで行っています。
(評価の原理は調べ切れていないため、まだよくわかっていません)

まずトップKの対象となるアイテムの候補一覧を取得します。

In [9]:
candidate_features = unique_rows_by_features(train, Tags.ITEM, Tags.ITEM_ID)
candidate_features.head()

,TE_movieId_rating,movieId,genres
262,1.465964,3,"[3, 4]"
18,1.703433,4,"[5, 9, 17, 7]"
552,1.374078,5,"[5, 9, 4, 7, 13]"
204,0.832853,6,"[5, 9, 8, 7, 13]"
808,0.306010,7,"[5, 9, 7]"


次にTwo Towerモデルと候補一覧から、トップK(K=20)のモデルを生成します。

実行に少々時間がかかるのと、情報や警告のログがでることに留意してください。

In [10]:
topk = 20
topk_model = model.to_top_k_encoder(candidate_features, k=topk, batch_size=128)

2023-11-01 07:11:07.221831: I tensorflow/core/common_runtime/executor.cc:1209] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs_3' with dtype int32 and shape [?]
	 [[{{node inputs_3}}]]
2023-11-01 07:11:07.238880: I tensorflow/core/common_runtime/executor.cc:1209] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs_3' with dtype int32 and shape [?]
	 [[{{node inputs_3}}]]
2023-11-01 07:11:07.745478: I tensorflow/core/common_runtime/executor.cc:1209] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs_1' with dtype int32 and shape [?]
	 [[{{node inputs_1}}]]
2023-11-01 07:11

INFO:tensorflow:Assets written to: /tmp/tmpuu5izfj8/assets


INFO:tensorflow:Assets written to: /tmp/tmpuu5izfj8/assets


In [11]:
topk_model.compile(run_eagerly=False)

検証用のデータセットを読み込むローダーを指定します。

In [12]:
eval_loader = mm.Loader(valid, batch_size=1024).map(mm.ToTarget(valid.schema, "movieId"))

ローダーとトップKモデルを用いて、評価しその結果を得ます。

評価値のうち `loss` は小さいほど良いモノです。
(他の値については別途調査が必要)

In [13]:
metrics = topk_model.evaluate(eval_loader, return_dict=True)
metrics

2023-11-01 07:11:24.168259: I tensorflow/core/common_runtime/executor.cc:1209] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


196/196 [==============================] - 6s 22ms/step - loss: 0.0698 - recall_at_10: 0.0116 - mrr_at_10: 0.0032 - ndcg_at_10: 0.0051 - map_at_10: 0.0032 - precision_at_10: 0.0012 - regularization_loss: 0.0000e+00 - loss_batch: 0.0695


{'loss': 0.06978823989629745,
 'recall_at_10': 0.011582625098526478,
 'mrr_at_10': 0.003222384722903371,
 'ndcg_at_10': 0.005127827636897564,
 'map_at_10': 0.003222384722903371,
 'precision_at_10': 0.0011582624865695834,
 'regularization_loss': 0.0,
 'loss_batch': 0.032966624945402145}

次にユーザーで問合せ、どのような作品が進められているかを確認します。

検証用のデータを16個(≒1バッチ)ずつ読み込むローダーを作成します。

In [14]:
eval_loader2 = mm.Loader(valid, batch_size=16, shuffle=False)
batch = next(iter(eval_loader2))

バッチ内のユーザーIDを表示します。

In [15]:
batch[0]['userId']

<tf.Tensor: shape=(16,), dtype=int32, numpy=
array([  30,  391, 4247,  626, 4151,   26, 2093,  457, 2164, 3156,   18,
        569,  475, 1566, 2045,  116], dtype=int32)>

バッチの各ユーザーに対してトップKを求めます。
これは各ユーザに対してオススメの映画を20個ずつリコメンドすることに相当します。

出力内容は最初の16x20個のデータはスコアで、次の16x20個は映画のIDです。

In [16]:
topk_model(batch[0])

TopKPrediction(scores=<tf.Tensor: shape=(16, 20), dtype=float32, numpy=
array([[5.2473106, 5.0404944, 5.021609 , 5.0208325, 5.009865 , 5.0030236,
        4.9795475, 4.9742455, 4.973768 , 4.958992 , 4.9535036, 4.9525466,
        4.929915 , 4.9290137, 4.9271564, 4.9243994, 4.920073 , 4.912663 ,
        4.911462 , 4.8900456],
       [5.851725 , 5.7771554, 5.7628655, 5.740063 , 5.7249045, 5.719814 ,
        5.699365 , 5.682789 , 5.6779604, 5.6580825, 5.6551504, 5.6478696,
        5.625829 , 5.6184826, 5.6140976, 5.610646 , 5.6096315, 5.607564 ,
        5.604478 , 5.6042905],
       [6.4238725, 6.2420106, 6.2263813, 6.20708  , 6.2034683, 6.193484 ,
        6.186962 , 6.1764736, 6.151105 , 6.1011386, 6.0965314, 6.072129 ,
        6.0611043, 6.059507 , 6.0534053, 6.039801 , 6.0396924, 6.039053 ,
        6.035053 , 6.022934 ],
       [5.637414 , 5.612073 , 5.5893292, 5.5749154, 5.5620933, 5.56049  ,
        5.5600495, 5.533441 , 5.5235496, 5.493829 , 5.4888487, 5.4654264,
        5.4516644, 5.

## Embeddingsの計算

いよいよTwo TowerモデルからユーザーとアイテムのEmbeddingを求めていきましょう。

まずはTwo Towerモデルからクエリーエンコーダー(≒ユーザ情報のエンコーダー)を取り出し、ファイルに保存します。

In [17]:
query_tower = model.query_encoder
query_tower.save(os.path.join(DATADIR, "query_tower"))

INFO:tensorflow:Assets written to: ./_data/query_tower/assets


INFO:tensorflow:Assets written to: ./_data/query_tower/assets


ファイルに保存したクエリーエンコーダーを読み込みます。
これにより一度学習が終了したエンコーダーはそれだけを抜き出して
Embeddingsの作成に再利用できます。

なおこのクエリエンコーダー(ユーザータワー)は永続化のデモのために作ったもので、
以降のEmbeddings作成には利用しません。

In [18]:
query_tower_loaded = tf.keras.models.load_model(os.path.join(DATADIR, "query_tower"))

学習に用いたデータからアイテム≒映画情報だけをすべて取り出し、
Two Towerモデルを使って候補Embeddingsを作成します。

In [19]:
item_features = (
    unique_rows_by_features(train, Tags.ITEM, Tags.ITEM_ID).compute().reset_index(drop=True)
)

In [20]:
item_embs = model.candidate_embeddings(Dataset(item_features, schema=train.schema.select_by_tag(Tags.ITEM)), 
                                       batch_size=1024, index=Tags.ITEM_ID)

2023-11-01 07:11:37.416231: I tensorflow/core/common_runtime/executor.cc:1209] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs_3' with dtype int32 and shape [?]
	 [[{{node inputs_3}}]]
2023-11-01 07:11:37.433294: I tensorflow/core/common_runtime/executor.cc:1209] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs_3' with dtype int32 and shape [?]
	 [[{{node inputs_3}}]]
2023-11-01 07:11:37.912718: I tensorflow/core/common_runtime/executor.cc:1209] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs_1' with dtype int32 and shape [?]
	 [[{{node inputs_1}}]]
2023-11-01 07:11

INFO:tensorflow:Assets written to: /tmp/tmp62231o2q/assets


INFO:tensorflow:Assets written to: /tmp/tmp62231o2q/assets


In [21]:
item_embs_df = item_embs.compute(scheduler="synchronous")

得られたアイテム(映画作品)ごとのEmbeddingsはこんな感じになります。

In [22]:
item_embs_df

,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
movieId,,,,,,,,,,,,,,,,,,,,,
3,-0.147444,0.050582,0.668026,-0.219845,-0.851177,0.134543,0.322166,0.464454,0.146026,0.443802,...,0.657453,-0.406107,0.109786,-0.061233,0.271269,-0.446135,0.066186,-0.217516,-0.042538,0.081565
4,-0.171692,-0.159529,0.876774,0.114507,-0.776455,0.065702,0.281130,0.627219,0.283734,0.273224,...,0.850813,-0.311217,0.045049,-0.238870,0.512616,-0.269778,0.138210,-0.296963,-0.075073,-0.050319
5,-0.136802,-0.072625,0.786183,0.015568,-0.745793,0.083267,0.296073,0.583151,0.283043,0.262127,...,0.779731,-0.326789,0.044756,-0.238565,0.508439,-0.285793,0.191447,-0.315846,-0.027651,0.008379
6,0.006336,0.009275,0.682236,-0.062951,-0.621638,0.154536,0.286177,0.471288,0.293091,0.220690,...,0.688260,-0.264250,-0.033298,-0.206060,0.470009,-0.282073,0.208179,-0.414836,0.037477,0.006867
7,0.209664,0.042268,0.611571,-0.134667,-0.459267,0.202502,0.248140,0.364766,0.282204,0.159699,...,0.585800,-0.154604,-0.070853,-0.202419,0.423178,-0.257228,0.190183,-0.435405,0.138830,-0.010245
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3670,0.487856,-0.537731,0.279271,-0.144404,-0.222311,0.324687,0.373445,0.061410,0.536197,-0.019739,...,0.652259,-0.173214,-0.073677,-0.144293,0.665965,-0.338501,0.336361,-0.390929,-0.171756,0.111921
3671,0.162033,0.404756,0.344630,-0.510022,-0.474333,0.218543,0.417738,0.059372,0.219341,0.287300,...,0.400295,-0.296371,0.056520,-0.137960,0.282014,-0.383754,0.364537,-0.311228,0.026025,0.346547
3672,0.295873,0.247074,0.335913,-0.425625,-0.380452,0.232765,0.483196,-0.028764,0.175907,0.206177,...,0.417341,-0.192952,0.131833,-0.159033,0.238903,-0.425992,0.373591,-0.255950,-0.020497,0.388377


以下ではユーザー情報に対して同じようにして、
ユーザーごとのEmbeddingsを取得し、表示しています。

In [23]:
user_features = (
    unique_rows_by_features(train, Tags.USER, Tags.USER_ID).compute().reset_index(drop=True)
)

In [24]:
user_embs = model.query_embeddings(Dataset(user_features, schema=train.schema.select_by_tag(Tags.USER)), 
                                       batch_size=1024, index=Tags.USER_ID)

INFO:tensorflow:Assets written to: /tmp/tmp5akawhqe/assets


INFO:tensorflow:Assets written to: /tmp/tmp5akawhqe/assets


In [25]:
user_embs_df = user_embs.compute(scheduler="synchronous")

In [26]:
user_embs_df

,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
userId,,,,,,,,,,,,,,,,,,,,,
3,0.457347,-0.032113,-0.161887,-0.364481,-0.031274,0.146369,0.511729,-0.334058,-0.281635,0.033677,...,0.067177,-0.061617,0.460160,-0.242294,0.108161,-0.461572,0.478589,0.080709,-0.239934,0.522792
4,0.304047,-0.176855,-0.348314,-0.362473,0.097906,0.023264,0.290237,-0.391030,0.454480,0.099648,...,0.315880,-0.025935,0.538470,-0.072790,0.436326,-0.390708,0.740093,-0.325449,-0.165854,0.350934
5,0.234984,0.042859,0.025779,-0.377027,0.034543,0.034160,0.437153,-0.125268,-0.112712,-0.062019,...,-0.141724,-0.117448,0.535965,-0.151354,0.103121,-0.265549,0.633049,0.077487,-0.230554,0.217161
6,0.510149,-0.253637,-0.265680,-0.343176,-0.010996,0.232242,0.308537,-0.146164,-0.129688,-0.050639,...,0.076357,-0.181547,0.206294,-0.260557,0.271412,-0.405505,0.430468,0.101922,0.019822,0.421268
7,0.337627,-0.185552,-0.126563,-0.490215,-0.140354,0.284786,0.259011,-0.084307,-0.102351,0.039404,...,0.192136,-0.154326,0.249652,-0.473305,0.440182,-0.452662,0.667182,0.000998,0.028943,0.544162
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6038,0.133999,-0.031560,-0.029437,-0.382227,0.002695,0.004241,0.172209,0.100258,-0.277449,0.083486,...,0.012242,-0.059664,0.101092,-0.175009,-0.009049,-0.424664,0.167673,-0.087562,0.153424,0.041428
6039,-0.413507,0.084532,0.557934,-0.402851,-0.423542,0.078683,0.470944,0.032453,0.426948,0.148860,...,0.535998,-0.047162,-0.577307,-0.096690,0.248784,-0.341795,-0.308271,-0.598545,-0.375325,0.452110
6040,0.197658,0.068413,0.271072,-0.219338,-0.015051,-0.052170,0.217175,0.295712,-0.050240,-0.157143,...,0.115599,-0.305095,0.220936,-0.306123,0.277484,-0.139658,0.582413,0.059640,0.171803,0.035237


---

以上でTwo Towerモデルの学習と、それを用いたEmbeddingsの計算のデモは終わりです。
お疲れさまでした。